# CDM IIIF Image Download

This script downloads images from Temple's digital collections.

# To Dos

Improve file naming conventions  
Synchronize metadata download with filenames  

Improve script efficiency  
Switch from urllib to requests  

# Install Packages


In [4]:
import urllib.request
import requests
import csv
import json
import pandas as pd
from google.colab import files
%load_ext google.colab.data_table

# 1) Download Temple's Collection Manifest 

This manifest includes all info on Temple's CDM IIIF images

Select a Collection @id below to download it's relevant Images and Metadata

In [5]:
# Provide CDM IIIF Manifest URL for Temple's digital collections
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/manifest.json'

resp = urllib.request.urlopen(manifestUrl)
data = resp.read().decode("utf-8")
data = json.loads(data)

df = pd.json_normalize(data['collections'])
del df['@type']
df

,@id,label
0,https://cdm16002.contentdm.oclc.org/iiif/info/...,Allied Posters of World War I
1,https://cdm16002.contentdm.oclc.org/iiif/info/...,Ambler Campus History in Photographs
2,https://cdm16002.contentdm.oclc.org/iiif/info/...,City Parks Association Photographs
3,https://cdm16002.contentdm.oclc.org/iiif/info/...,Frank G. Zahn Railroad Photograph Collection
4,https://cdm16002.contentdm.oclc.org/iiif/info/...,Franklin H. Littell Papers
5,https://cdm16002.contentdm.oclc.org/iiif/info/...,George D. McDowell Philadelphia Evening Bullet...
6,https://cdm16002.contentdm.oclc.org/iiif/info/...,George D. McDowell Philadelphia Evening Bullet...
7,https://cdm16002.contentdm.oclc.org/iiif/info/...,Housing Association of the Delaware Valley Pho...
8,https://cdm16002.contentdm.oclc.org/iiif/info/...,Impulse Dance Annuals
9,https://cdm16002.contentdm.oclc.org/iiif/info/...,Jacob H. Gomborow Papers


## Create function to pull images from URLs

In [6]:
pid = 0
img_urls = {}
img_smalls = {}


In [7]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 2) Download Collection-Level Metadata

Make this do iterative downloads for all collections

In [8]:
# provide the url to the collection
collectionURL = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/manifest.json"

In [9]:
resp1 = urllib.request.urlopen(collectionURL)
data1 = resp1.read().decode("utf-8")
data2 = json.loads(data1)
collection = data2['first']['@id']
responseManifest = requests.get(collection)
collManifest = json.loads(responseManifest.text)

In [10]:
csv_out = csv.writer(open('collection.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])

44

In [11]:
for i in collManifest['manifests']:
    title = date = contributor = description = subject = ''
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)
    title = manifestItem['metadata'][0]['value']
    date = manifestItem['metadata'][1]['value']
    contributor = manifestItem['metadata'][2]['value']
    description = manifestItem['metadata'][3]['value']
    subject = manifestItem['metadata'][4]['value']
    csv_out.writerow([title, date, contributor, description, subject])    

In [12]:
df1 = pd.read_csv('collection.csv')
df1

,Title,Date,Contributor,Description,Subject
0,Sottoscrivete al prestito,[ca. 1917],"Capranesi, Giovanni, 1852-1921, artist.; Istit...",Allegorical depiction of Italy as a female wea...,"Date of publication from Rawls, Walton, Wake u..."
1,Step into your place,Poster (Great Britain. Parliamentary Recruitin...,1915,Great Britain. Parliamentary Recruiting Commit...,British recruiting poster. Illustration depic...
2,Indian Ink,1916,"Pereira, Harold B., artist.;",Poster advertising the Christmas 1916 edition ...,"Signed: Harold B. Pereira, 16 [i.e. 1916?]. Or..."
3,Per la libertà e la civiltà del mondo,1918,"Dudovich, Marcello, 1878-1962, artist.; Atelie...",Italian poster urging subscription to the nati...,Signed: M. Dudovich. Date of publication from ...
4,The watchers of the seas,[ca. 1914-1918],"Raven-Hill, L. (Leonard), 1867-1942, artist.; ...",British naval recruiting poster that asks for ...,"Signed: ""L. Ravenhill."" Date based on the date..."
...,...,...,...,...,...
987,There is still a place in the line for you,Poster (Great Britain. Parliamentary Recruitin...,1915,"Haycock-Cadle Co., printer.; Great Britain. Pa...",Text says that there is still a place in the l...
988,Think!,Poster (Great Britain. Parliamentary Recruitin...,[ca. 1914-1918],"Oakley, H., artist.; Chorley & Pickersgill Ltd...",Text asks if you are content to let others fig...
989,Help to end the war,Poster (Great Britain. Parliamentary Recruitin...,1915,"David Allen & Sons, printer.; Great Britain. P...",Text says to help to win the war by enlisting ...
990,Help to end the war,Poster (Great Britain. Parliamentary Recruitin...,[ca. 1914-1918],"Roberts & Leet, printer.; Great Britain. Parli...",Text says to help to win the war by enlisting ...


In [1]:
files.download('collection.csv')

NameError: ignored

# Iterate and get all metadata csv files

Close to working.

In [ ]:
def downallmetadata(df, pid = 0):
  pid = pid + 1   
  collection_title = str(pid) + '.csv'
  csv_out = csv.writer(open(collection_title, 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])
  for collectionUrl in df['@id']:
    resp1 = urllib.request.urlopen(collectionUrl)
    data1 = resp1.read().decode("utf-8")
    data2 = json.loads(data1)
    collection = pd.json_normalize(data2['first'])
    for url in collection['@id']:
      resp2 = urllib.request.urlopen(url)
      data3 = resp2.read().decode("utf-8")
      data4 = json.loads(data3)
      for i in data4['manifests']:
        title = date = contributor = description = subject = ''
        manifestItemUrl = i['@id']
        responseManifestItem = requests.get(manifestItemUrl)
        manifestItem = json.loads(responseManifestItem.text)
        title = manifestItem['metadata'][0]['value']
        date = manifestItem['metadata'][1]['value']
        contributor = manifestItem['metadata'][2]['value']
        description = manifestItem['metadata'][3]['value']
        subject = manifestItem['metadata'][4]['value']
        csv_out.writerow([title, date, contributor, description, subject]) 
             

In [ ]:
downallmetadata(df)

KeyboardInterrupt: ignored

# 3) Download Images from specific Collection

This will download images from whichever collection you selected above

It is currently set to stop after 30 images, but you can extend how many images it downloads

Actually it's not limiting number of items download, maybe retry

In [13]:
def iiif_download(collectionURL, pid = 0):
  resp1 = urllib.request.urlopen(collectionURL)
  data1 = resp1.read().decode("utf-8")
  data2 = json.loads(data1)
  collection = pd.json_normalize(data2['first'])
  for url in collection['@id']:
    resp2 = urllib.request.urlopen(url)
    data3 = resp2.read().decode("utf-8")
    data4 = json.loads(data3)
    item = pd.json_normalize(data4['manifests'])
    for imageurl in item['@id']:
      resp3 = urllib.request.urlopen(imageurl)
      data5 = resp3.read().decode("utf-8")
      image = json.loads(data5)
      for jpg in image['sequences'][0]['canvases']:
          img_urls[pid]=jpg['images'][0]['resource']['service']['@id']
          filename = str(pid) + ".jpg"
          #imagename[pid] = item['label']
          #imagefile = str(imagename[pid]).replace(" ","").lower()
          # print(imagefile)
          #filename = imagefile[0:10] + ".jpg"
          turl = img_urls[pid]+"/full/pct:100/0/default.jpg"
          urllib.request.urlretrieve(turl, filename)
          #files.download(filename)
          pid = pid +1
          if pid == 30:
                break

In [ ]:
iiif_download(collectionURL)

In [ ]:
!zip -r /content/IIIF.zip /content/

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/active_config (stored 0%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2022.01.07/ (stored 0%)
updating: content/.config/logs/2022.01.07/14.32.57.118850.log (deflated 53%)
updating: content/.config/logs/2022.01.07/14.33.42.565580.log (deflated 53%)
updating: content/.config/logs/2022.01.07/14.33.22.232212.log (deflated 54%)
updating: content/.config/logs/2022.01.07/14.33.41.864886.log (deflated 54%)
updating: content/.config/logs/2022.01.07/14.33.15.504888.log (deflated 86%)
updating: content/.config/logs/2022.01.07/14.32.37.114755.log (deflated 91%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/configurations/ (stored 0%)
updating: content/.config/configurations/config_default (deflated 15%)
updating: content/.config/.last_u

In [ ]:
files.download('/content/IIIF.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Download all TU CDM IIIF images

This function currently will get all images from all collections in Temple's ContentDM IIIF database. Not really usable as such

I haven't created yet an iterative downloader of all metadata for each collection, but that could be next

In [ ]:
def downthemall(df, pid = 0):
  for collectionUrl in df['@id']:
    resp1 = urllib.request.urlopen(collectionUrl)
    data1 = resp1.read().decode("utf-8")
    data2 = json.loads(data1)
    collection = pd.json_normalize(data2['first'])
    for url in collection['@id']:
      resp2 = urllib.request.urlopen(url)
      data3 = resp2.read().decode("utf-8")
      data4 = json.loads(data3)
      item = pd.json_normalize(data4['manifests'])
      for imageurl in item['@id']:
        resp3 = urllib.request.urlopen(imageurl)
        data5 = resp3.read().decode("utf-8")
        image = json.loads(data5)
        for jpg in image['sequences'][0]['canvases']:
            img_urls[pid]=jpg['images'][0]['resource']['service']['@id']
            filename = str(pid) + ".jpg"
            #imagename[pid] = item['label']
            #imagefile = str(imagename[pid]).replace(" ","").lower()
            # print(imagefile)
            #filename = imagefile[0:10] + ".jpg"
            turl = img_urls[pid]+"/full/pct:100/0/default.jpg"
            urllib.request.urlretrieve(turl, filename)
            files.download(filename)
            pid = pid +1
            if pid == 100:
                  break

In [ ]:
downthemall(df)

# Classify Images

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')



There are various ways in which we could load this type of data using fastai. In this example we'll use ImageDataLoaders.from_csv. As the name suggests the from_csv method of ImagDataLoaders loads data from a CSV file. We need to tell fastai a few things about how to load the data to use this method:

    The path to the folder where images and CSV file are stored.
    The coloumns in the CSV file which contain the labels.
    One 'item transform' Resize() to resize all the images to a standard size.

We will go into more detail later in the lesson. We'll create a variable ad_data which will be used to store the steps for loading this data:


In [ ]:
# Import image data

ad_data = ImageDataLoaders.from_csv(
    path="../input/computer-vision-for-the-humanities-ph/ads_data/",  # root path to csv file and image directory
    csv_fname="ads_upsampled.csv",  # the name of our csv file
    folder="images/",  # the folder where our images are stored
    fn_col="file",  # the file column in our csv
    label_col="label",  # the label column in our csv
    item_tfms=Resize(224, ResizeMethod.Squish),  # resize imagesby squishing so they are 224x224 pixels
    seed=42,  # set a fixed seed to make results more reproducible
)

ad_data.show_batch()

In [ ]:
#prep model
learn = cnn_learner(
    ad_data,  # the data the model will be trained on
    resnet18,  # the type of model we want to use
    metrics=accuracy,  # the metrics to track
)